In [2]:
import pyspark

ImportError: No module named pyspark

In [1]:
import pyspark
from collections import defaultdict

sc = pyspark.SparkContext()

lines = sc.textFile('/Users/idzhang/COLLEGENOW/CS205/100.txt')
words = lines.flatMap(lambda line: line.split(' '))

# Filter out any words that contain only numbers, contain only letters which are capitalized, or contains
# only letters which are capitalized and ends with a period.
# Be sure that filtering doesn't change ordering of words!
words = words.filter(lambda word: not (word.isupper() or (word[:-1].isupper() and word[-1] == '.') or word.isdigit())) 
numWords = words.count()
wordsList = words.collect()

# Map words to three word phrases
wordsWithIdx = words.zipWithIndex()

def mapWordToPhrase(w, wordsList):
	(word, idx) = w
	if (idx < numWords - 2):
		return [(word, wordsList[idx+1]), [(wordsList[idx+2], 1)]]
	else:
		return []

# Phrases contains ((word1, word2), [word3]) for every 3 word phrase
phrases = wordsWithIdx.flatMap(lambda w: mapWordToPhrase(w, wordsList)).cache()

# threeWords contains ((word1, word2), [all possible word3s in a list with duplicates])
threeWords = phrases.reduceByKey(lambda x,y: x+y)

ImportError: No module named pyspark

In [ ]:
def reducesWordList(wordList):
	d = 
	wordList

# Reduce by key (first 2 words of 3 word phrase)
twoWords = threeWords.mapValues(reducesWordList)

# # Perform map (word -> (word,1)) and then reduce (add counts) operations on 3-word phrases
# # twoWords = twoWords.mapValues(map(lambda word: (word, 1), val))
# twoWords = twoWords.mapValues(map(lambda x: (x,1), words)).mapValues()

print threeWords.take(15)